In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


# Read the data for January. How many columns are there?

In [5]:
df_jan.shape[1]

19

In [6]:
df_jan.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

# What's the standard deviation of the trips duration in January? 

In [7]:
# Compute the duration in minutes
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

# Calculate the standard deviation of trip durations
std_duration = df_jan['duration'].std()

print(f'The standard deviation of trip durations is {std_duration:4f} minutes.')

The standard deviation of trip durations is 42.594351 minutes.


# What fraction of the records left after you dropped the outliers?

In [8]:
# Define the duration range to keep (1 to 60 minutes inclusive)
min_duration = 1
max_duration = 60

# Filter the DataFrame to keep only records with duration between 1 and 60 minutes
filtered_df = df_jan[(df_jan['duration'] >= min_duration) & (df_jan['duration'] <= max_duration)]

# Calculate the fraction of records remaining after removing outliers
remaining_records = len(filtered_df) / len(df_jan)

print(f'The fraction of records left after removing outliers is {remaining_records:.4f}')

The fraction of records left after removing outliers is 0.9812


# What's the dimensionality of this matrix (number of columns)?

In [9]:
# Filter the DataFrame to keep only records with duration between 1 and 60 minutes
df_jan = df_jan[(df_jan['duration'] >= min_duration) & (df_jan['duration'] <= max_duration)]

# Select only the pickup and dropoff location ID columns and cast them to strings
df_jan['PULocationID'] = df_jan['PULocationID'].astype(str)
df_jan['DOLocationID'] = df_jan['DOLocationID'].astype(str)

# Convert DataFrame to a list of dictionaries
train_dicts = df_jan[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Get the dimensionality of the feature matrix
print(f'The dimensionality of the feature matrix is {X_train.shape[1]} columns.')

The dimensionality of the feature matrix is 515 columns.


# What's the RMSE on train?

In [10]:
# Define the target variable
y_train = df_jan['duration'].values

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the durations on the training data
y_pred = model.predict(X_train)

# Calculate the RMSE on the training data
rmse = mean_squared_error(y_train, y_pred, squared=False)

print(f'The RMSE on the training data is {rmse:.4f}')

The RMSE on the training data is 7.6493


# What's the RMSE on validation? 

In [11]:
# Compute the duration in minutes
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter the DataFrame to keep only records with duration between 1 and 60 minutes
df_feb = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]

# Select only the pickup and dropoff location ID columns and cast them to strings
df_feb['PULocationID'] = df_feb['PULocationID'].astype(str)
df_feb['DOLocationID'] = df_feb['DOLocationID'].astype(str)

# Convert DataFrame to a list of dictionaries
dicts_feb = df_feb[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transform the February data using the fitted dictionary vectorizer
X_feb = dv.transform(dicts_feb)
y_feb = df_feb['duration'].values

# Predict the durations for the February data
y_pred_feb = model.predict(X_feb)

# Calculate the RMSE on the validation data
rmse_feb = mean_squared_error(y_feb, y_pred_feb, squared=False)
print(f'The RMSE on the validation data (February 2023) is {rmse_feb:.4f}')

The RMSE on the validation data (February 2023) is 7.8118
